In [1]:
%cd /content/drive/MyDrive/DeepLearning_HW2

/content/drive/MyDrive/DeepLearning_HW2


In [2]:
!ls

all_plants_data  archive.zip  hw#2  Plants_2


In [3]:
# %cd ./Plants_2/

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import os
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, transforms
from torch.utils import data
from torch.autograd import Variable

In [3]:
IMG_WIDTH = 224  # width of image
IMG_HEIGHT = 224  # height of image
# BATCH_SIZE = 32  # batch size
# SEED = 0  # random seed
# DROPOUT = 0.5  # dropout probability
# LEARNING_RATE = 0.001    # learning rate
# NUM_EPOCHS = 10   # number of epochs
# CNN_MODEL_PATH = 'cnn_classification_model.pt'  # path for saved CNN model
# NUM_CLASSES = 2 

In [4]:
transformation = transforms.Compose([
        # resize
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        # transform to tensors
        transforms.ToTensor(),
        # normalize
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [5]:


train_folder = "./Plants_2/train/"
train_dataset = datasets.ImageFolder(root = train_folder, 
                transform = transformation)

train_loader = data.DataLoader(train_dataset, batch_size = 8, shuffle = True)

In [6]:
val_folder = "./Plants_2/valid/"
val_dataset = datasets.ImageFolder(root = val_folder, 
                transform = transformation)

val_loader = data.DataLoader(val_dataset, batch_size = 8, shuffle = True)

In [7]:
test_folder = "./Plants_2/test/"
test_dataset = datasets.ImageFolder(root = test_folder, 
                transform = transformation)

test_loader = data.DataLoader(test_dataset, batch_size = 8, shuffle = True)

In [11]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),           
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),           
            nn.ReLU(),
			nn.Conv2d(24, 48, 4, stride=2, padding=1),           
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Tanh(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [12]:
def create_model():
    autoencoder = Autoencoder()
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder

def print_model(encoder, decoder):
    print("============== Encoder ==============")
    print(encoder)
    print("============== Decoder ==============")
    print(decoder)
    print("")

In [13]:
autoencoder = create_model()

============== Encoder ==============
Sequential(
  (0): Conv2d(3, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(12, 24, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): Conv2d(24, 48, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (5): ReLU()
)
============== Decoder ==============
Sequential(
  (0): ConvTranspose2d(48, 24, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): ConvTranspose2d(24, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): ConvTranspose2d(12, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (5): Tanh()
)

Model moved to GPU in order to speed up training.


In [14]:
def get_torch_vars(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [ ]:
# criterion = nn.BCELoss()
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters())

for epoch in range(100):
    running_loss = 0.0
    for i, (inputs, _) in enumerate(train_loader, 0):
        inputs = get_torch_vars(inputs)
        # ============ Forward ============
        encoded, outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        # ============ Backward ============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # ============ Logging ============
        running_loss += loss.data
        if i % 5 == 4:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,     5] loss: 0.003
[1,    10] loss: 0.002
[1,    15] loss: 0.002
[1,    20] loss: 0.001
[1,    25] loss: 0.001
